# Benchmarking Our Algorithm against others

In [11]:
import sys
sys.path.append('../../')
from time import time
import signal
from os import listdir
import numpy as np
import pickle as pkl


from panav.sequential import sequential_planning
from panav.SAMP.solvers import Tube_Planning, Simple_MILP_Planning

from benchmarks.ST_RRTStar_wrapper import sequential_ST_RRTStar 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
def get_env_name(filename):
    fs = filename.split('.')
    return fs[0]


def run_experiments(env_name, 
                    env_info, 
                    N_range, 
                    algs, 
                    TIMEOUT = 100,
                    results = [], 
                    executed = None):
    
    '''
        Checkpointed results and timeouted can be passed in so we can resume operation.
    '''

    if executed is None:
        executed = {alg: {N :False for N in N_range} for alg in algs}

    for N in N_range: 
        # Construct the environment
        constructor = env_info['constructor']
        specs = env_info['specs']
        env = constructor(**specs,N_agent = N)

        # Run the experiment on each algorithm
        for name,alg in algs.items():

            if executed[name][N]:
                continue
            
            print("Env:", env_name, "Alg:", name, "N agent:",N)

            result = {}
            
            def timeout_handler(signum,frame):
                print("Algorithm timeout")
                # Mark the rest of the executed table as True, so we don't ever visit this algorithm again
                for n in executed[alg]:
                    if n>=N:
                        executed[alg][n] = True
                        
                raise TimeoutError("Timeout")
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                t0 = time()
                plan = alg(env)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["env"]=env

            results.append(result)

            if t>TIMEOUT:
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                result['plan'] = plan
                print("On time for algorithm", name, "N agent = ", N)

        # Save checkpoint data
        with open("./data/{}.pkl".format(env_name),'wb') as fp:
            pkl.dump(results,fp)



In [27]:
bloating_r = 0.5
vmax = 1.0

# Specify the Algorithms
algs = {  
        "Seq_Tube": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        "Seq_Tube_nonLazy": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),      
        "Seq_S2M2": lambda env: sequential_planning(Simple_MILP_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),
        "ST-RRT*": lambda env: sequential_ST_RRTStar(env,vmax,bloating_r)
        }
# Load environment presets
env_names = [get_env_name(fn) for fn in listdir('./envs')]

env_info = {}

for name in env_names:
    with open(f'./envs/{name}.pkl','rb') as fp:
        env_info[name] = pkl.load(fp)

In [33]:
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
env_name = 'MultiTunnel'
max_N = 40

N_range = np.arange(1,max_N+1)
results = []
executed = {alg: {N :False for N in N_range} for alg in algs}

In [34]:
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                100,
                results,
                executed)

Env: MultiTunnel Alg: Seq_Tube N agent: 1
Planning for agent 0/1
On time for algorithm Seq_Tube N agent =  1
Env: MultiTunnel Alg: Seq_Tube_nonLazy N agent: 1
Planning for agent 0/1
On time for algorithm Seq_Tube_nonLazy N agent =  1
Env: MultiTunnel Alg: Seq_S2M2 N agent: 1
Planning for agent 0/1
On time for algorithm Seq_S2M2 N agent =  1
Env: MultiTunnel Alg: ST-RRT* N agent: 1
Planning for agent 0/1
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


Info:    SpaceTimeRRT: Created 133 states (45 start + 88 goal)
On time for algorithm ST-RRT* N agent =  1
Env: MultiTunnel Alg: Seq_Tube N agent: 2
Planning for agent 0/2
Planning for agent 1/2
On time for algorithm Seq_Tube N agent =  2
Env: MultiTunnel Alg: Seq_Tube_nonLazy N agent: 2
Planning for agent 0/2
Planning for agent 1/2
On time for algorithm Seq_Tube_nonLazy N agent =  2
Env: MultiTunnel Alg: Seq_S2M2 N agent: 2
Planning for agent 0/2
Planning for agent 1/2
On time for algorithm Seq_S2M2 N agent =  2
Env: MultiTunnel Alg: ST-RRT* N agent: 2
Planning for agent 0/2
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 1/2Info:    SpaceTimeRRT: Created 117 states (52 start + 65 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
On time for algorithm ST-RRT* N agent =  2
Info:    SpaceTimeRRT: Created 72 states (34 start + 38 goal)
Env: MultiTunnel Alg: Seq_Tube N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube N agent =  3
Env: MultiTunnel Alg: Seq_Tube_nonLazy N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube_nonLazy N agent =  3
Env: MultiTunnel Alg: Seq_S2M2 N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_S2M2 N agent =  3
Env: MultiTunnel Alg: ST-RRT* N agent: 3
Planning for agent 0/3
Debug:   SpaceTimeRRT: Planner range detected to be inf
Inf

...

Info:    SpaceTimeRRT: Created 52 states (29 start + 23 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 6/9
Info:    SpaceTimeRRT: Created 66 states (34 start + 32 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Info:    SpaceTimeRRT: Increased time bound factor to 4.00
Planning for agent 7/9
Info:    SpaceTimeRRT: Created 516 states (509 start + 7 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 8/9
Info:    SpaceTimeRRT: Created 55 states (29 start + 26 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


On time for algorithmInfo:    SpaceTimeRRT: Created 194 states (88 start + 106 goal)
 ST-RRT* N agent =  9
Env: MultiTunnel Alg: Seq_Tube N agent: 10
Planning for agent 0/10
Planning for agent 1/10
Planning for agent 2/10
Planning for agent 3/10
Planning for agent 4/10
Planning for agent 5/10
Planning for agent 6/10
Planning for agent 7/10
Planning for agent 8/10
Planning for agent 9/10
On time for algorithm Seq_Tube N agent =  10
Env: MultiTunnel Alg: Seq_Tube_nonLazy N agent: 10
Planning for agent 0/10
Planning for agent 1/10
Planning for agent 2/10
Planning for agent 3/10
Planning for agent 4/10
Planning for agent 5/10
Planning for agent 6/10
Planning for agent 7/10
Planning for agent 8/10
Planning for agent 9/10
On time for algorithm Seq_Tube_nonLazy N agent =  10
Env: MultiTunnel Alg: Seq_S2M2 N agent: 10
Planning for agent 0/10
Planning for agent 1/10
Planning for agent 2/10
Planning for agent 3/10
Planning for agent 4/10
Planning for agent 5/10
Planning for agent 6/10
Planning f

In [31]:
executed

{'Seq_Tube': {1: False,
  2: False,
  3: False,
  4: False,
  5: False,
  6: False,
  7: False,
  8: False,
  9: False,
  10: False,
  11: False,
  12: False,
  13: False,
  14: False,
  15: False,
  16: False,
  17: False,
  18: False,
  19: False,
  20: False,
  21: False,
  22: False,
  23: False,
  24: False,
  25: False,
  26: False,
  27: False,
  28: False,
  29: False,
  30: False,
  31: False,
  32: False,
  33: False,
  34: False,
  35: False,
  36: False,
  37: False,
  38: False,
  39: False,
  40: False},
 'Seq_Tube_nonLazy': {1: False,
  2: False,
  3: False,
  4: False,
  5: False,
  6: False,
  7: False,
  8: False,
  9: False,
  10: False,
  11: False,
  12: False,
  13: False,
  14: False,
  15: False,
  16: False,
  17: False,
  18: False,
  19: False,
  20: False,
  21: False,
  22: False,
  23: False,
  24: False,
  25: False,
  26: False,
  27: False,
  28: False,
  29: False,
  30: False,
  31: False,
  32: False,
  33: False,
  34: False,
  35: False,
  36: Fals